### Code to run pypsha and get output pgas
##### From Neetesh Sharma and modified by Emily Mongold and Neetesh Sharma

In [1]:
import numpy as np
import pandas as pd
import geopandas as gpd

from NNR import NNR

import pickle
import utm
import numpy.matlib as nm
import os
import datetime

Run the following only for the first run of pypsha:

In [9]:
def run_pypsha(site_file,nmaps,outdir):
    # have some fixed parameters, such as attenuations and PGA as IM
    test_site = psha.PSHASite(name = 'site',
                            site_filename = site_file,
                            erf=1, intensity_measures = [1],
                            attenuations = [1,2,4],
                            overwrite=True)
    test_site.write_opensha_input(overwrite = True)
    test_site.run_opensha(overwrite= True, write_output_tofile = True)
    event_set = psha.PshaEventSet(test_site)
    sa_intensity_ids = ["ASK2014_PGA","BSSA2014_PGA","CY2014_PGA"]
    event_set.generate_sa_maps(sa_intensity_ids, nmaps)
    with open(outdir + 'event_save_all_alameda.pickle','wb') as handle:
        pickle.dump(event_set, handle)
    return

In [27]:
file = "all_alameda_sites.csv"
site_file = os.path.join(os.getcwd(),file)
run_pypsha(site_file,2,'./')
with open('./event_save_all_alameda.pickle','rb') as handle:
    event_set = pickle.load(handle)
test_site = psha.PSHASite(name = 'alameda',
                            site_filename = site_file,
                            erf=1,
                            intensity_measures = [1],
                            attenuations = [1,2,4],
                            overwrite=True)
test_site.write_opensha_input(overwrite = True)
test_site.run_opensha(overwrite= True, write_output_tofile = True)
event_set = psha.PshaEventSet(test_site)
nmaps = 2 #per erf row
sa_intensity_ids = ["ASK2014_PGA","BSSA2014_PGA","CY2014_PGA"]

event_set.generate_sa_maps(sa_intensity_ids, nmaps)
with open('./event_save_all_alameda.pickle','wb') as handle:
    pickle.dump(event_set, handle)

Run starting here once event_save.pickle has been generated:

In [56]:
nsim = event_set.maps['ASK2014_PGA'].shape[0]/2

In [57]:
site_file = "./all_alameda_sites.csv"
with open('./event_save_all_alameda.pickle','rb') as handle:
    event_set = pickle.load(handle)


# generate random gmm from 0-2 for each event
gmm = np.array(np.random.choice([0, 1, 2], size=(nsim,)))
np.savetxt('gmm.csv', gmm, delimiter=",")

# once gmm is generated, load from here:
# gmm = np.loadtxt('gmm.csv',delimiter=',')


In [58]:
## code to save pgas_pypsha.csv for the new vs30 values
def run_pypsha_pgas(savedir,event_file,gmm):
    '''function to obtain pgas from pypsha output file based on gmm for each scenario.
    savedir is the directory where the output will be saved
    event_file is the im directory + 'event_save.pickle' unless event_file has a new name
    gmm is the random variable for the ground motion model for each scenario
    output is the text is saved in the im directory as 'pgas_pypsha.csv'
    '''
    
    with open(event_file,'rb') as handle:
        event_set = pickle.load(handle)
    
    sa_intensity_ids = ["ASK2014_PGA","BSSA2014_PGA","CY2014_PGA"] # should edit to obtain automatically
    ask_events = event_set.maps[sa_intensity_ids[0]]
    ask_events = ask_events.reset_index(level='map_id')
    ask_events = ask_events[ask_events['map_id'] == 0]
    ask_events.drop('map_id', axis=1,inplace=True)
    
    pga = np.zeros(ask_events.shape)
    n = 0
    for scen in event_set.maps[sa_intensity_ids[0]]['site0'].keys():
        if scen[-1] == 0:
            mod = int(gmm[n])
            pga[n,:] = event_set.maps[sa_intensity_ids[mod]].loc[scen]
            n += 1
            
    np.savetxt(savedir + 'pypsha_pgas.csv',pga,delimiter=',')
    return


In [71]:
run_pypsha_pgas('./',event_file='./event_save_all_alameda.pickle',gmm=gmm)

Run the following to get pgas at each grid location

In [100]:
pgas = {}
pgaZ = np.genfromtxt('./pypsha_pgas.csv', delimiter=',')
data = pd.read_csv(site_file,index_col=0)
lons = data['x'] # or lon
lats = data['y'] # or lat

nsim = len(pgaZ)

utmX = np.zeros(len(lons))
utmY = np.zeros(len(lons))
for i in range(len(utmX)):
    (utmX[i], utmY[i], reg,northrn) = utm.from_latlon(lats[i],lons[i])

X = (utmX - 558500)/1000  ## changed from 559000 [cut off lhs island]
Y = (utmY - 4172400)/1000

In [101]:
xs = range(117)
x = nm.repmat(xs,1,114)
indX = list(x[0])

indY = []
y = []
for i in range(114):
    y.append(np.full(117,i))
for i in range(len(y)):
    for j in list(y[i]):
        indY.append(j)
        
y_t = list(map(lambda x:x/10,indY))
x_t = list(map(lambda x:x/10,indX))
utmX = list(map(lambda x:(1000*x) + 558500,x_t))
utmY = list(map(lambda x:(1000*x) + 4172400,y_t))

grid = pd.DataFrame(columns = ['y','x'])
grid['y'] = y_t
grid['x'] = x_t
LAT = np.zeros(len(utmX))
LON = np.zeros(len(utmY))
for i in range(len(utmX)):
    lat,lon = utm.to_latlon(utmX[i], utmY[i], reg, northrn)
    LAT[i] = lat
    LON[i] = lon


grid['lat'] = LAT
grid['lon'] = LON
grid['indX'] = indX
grid['indY'] = indY

np.array(pgaZ.T).shape

(130, 2423)

In [111]:
nsim = pgaZ.shape[0]

Alameda = gpd.read_file('../alameda_plots/alameda_county.geojson')
for sim in range(nsim):
    Z = []
    for i in range(pgaZ.shape[1]): #pga at each location
        Z.append(pgaZ[sim,i])
    Z = np.array(Z)
    z_t = NNR(np.array([x_t,y_t]).T,np.array([X,Y]).T,Z,sample_size = -1, n_neighbors = 4, weight = 'distance2')
    grid['pga'] = z_t
    gdf = gpd.GeoDataFrame(grid, geometry=gpd.points_from_xy(grid['lon'], grid['lat']),crs = 'EPSG:4326')
    points = gpd.sjoin(gdf,Alameda)
    points.reset_index(inplace=True,drop = True)

    if sim == 0:
        for i in range(len(points)):
            pgas[i] = []
    for i in range(len(points)):
        pgas[i].append(points['pga'][i])
        
for i in range(len(points)):
    pgas[i] = np.array(pgas[i])
    

In [112]:
pd.DataFrame(pgas).to_csv('./pgas.csv')

In [3]:
## RUN JUST THIS CODE TO SAVE M TO CSV
with open('./event_save_all_alameda.pickle','rb') as handle:
    event_set = pickle.load(handle)
M = np.array(event_set.events.metadata['magnitude'])
np.savetxt('M.csv',M,delimiter=',')

In [7]:
## RUN JUST THE CPT POINTS AND SAVE THAT AS PGAS_CPT.CSV TO AVOID KEY ERROR

def setup_cpt(datadir):
    ''' function setup_cpt to load USGS cpt data from folder and output a dictionary
    input: datadir is the directory path to the cpt data
    output: cpt is a dictionary with an np directory for each borehole set of cpt data
    '''

    d = {}
    names = []
    cpt = {}
    # Constants
    g = 9.81
    Pa = 0.101325  # MPa
    rho_w = 1  # Mg/m^3
    gamma_w = rho_w * g / 1000  # MPa

    for filename in filter(lambda x: x[-4:] == '.txt', os.listdir(datadir)):

        with open(os.path.join(datadir, filename)) as f:
            name = datadir + filename
            df_temp = pd.read_csv(name, delimiter="\s+", skiprows=17)
            df_temp = df_temp.dropna(axis='columns', how='all')
            df_temp.columns = ['Depth', 'Tip_Resistance', 'Sleeve_Friction', 'Inclination', 'Swave_travel_time']
            df_temp = df_temp[-((df_temp['Sleeve_Friction'] < 0) | (df_temp['Tip_Resistance'] < 0))]
            df_temp = df_temp.reset_index(drop=True)
            df_temp = df_temp[df_temp['Depth'] <= 20]
            df_temp['Sleeve_Friction'] = df_temp['Sleeve_Friction'] / 1000  # convert to units of MPa

            temp = pd.DataFrame(np.zeros(shape=(len(df_temp), 7)),
                                columns=['start', 'q_c', 'f_s', 'd', 'dz', 'gamma', 'R_f'])
            temp['q_c'][0] = df_temp['Tip_Resistance'][0] / 2
            temp['f_s'][0] = df_temp['Sleeve_Friction'][0] / 2
            temp['d'][0] = np.average([temp['start'][0], df_temp['Depth'][0]])
            temp['dz'][0] = df_temp['Depth'][0] - temp['start'][0]
            temp['R_f'][0] = 100 * temp['f_s'][0] / temp['q_c'][0]
            temp['gamma'][0] = gamma_w * (0.27 * (np.log10(temp['R_f'][0])) +
                                          0.36 * np.log10(temp['q_c'][0] / Pa) + 1.236)
            for i in range(1, len(df_temp)):
                temp['start'][i] = df_temp['Depth'].iloc[i - 1]
                temp['f_s'][i] = np.average([df_temp['Sleeve_Friction'].iloc[i], df_temp['Sleeve_Friction'].iloc[i - 1]])
                temp['q_c'][i] = np.average([df_temp['Tip_Resistance'].iloc[i], df_temp['Tip_Resistance'].iloc[i - 1]])
                temp['d'][i] = np.average([temp['start'][i], df_temp['Depth'].iloc[i]])
                temp['dz'][i] = df_temp['Depth'].iloc[i] - temp['start'][i]
                temp['R_f'][i] = 100 * temp['f_s'][i] / temp['q_c'][i]
                # Calculating soil unit weight from Robertson and Cabal (2010)
                if temp['R_f'][i] == 0:
                    if temp['q_c'][i] == 0:
                        temp['gamma'][i] = gamma_w * 1.236
                    else:
                        temp['gamma'][i] = gamma_w * (0.36 * np.log10(temp['q_c'][i] / Pa) + 1.236)
                elif temp['q_c'][i] == 0:
                    temp['gamma'][i] = gamma_w * (0.27 * (np.log10(temp['R_f'][i])) + 1.236)
                else:
                    temp['gamma'][i] = gamma_w * (0.27 * np.log10(temp['R_f'][i]) +
                                                  0.36 * np.log10(temp['q_c'][i] / Pa) + 1.236)

            temp['dsig_v'] = temp['dz'] * temp['gamma']

            key = list(dict(l.strip().rsplit(maxsplit=1) for l in open(name) if any(l.strip().startswith(i) for i in 'File name:')).values())[0]
            names.append(key)
            d[key] = dict(l.strip().rsplit('\t', maxsplit=1) for l in open(name) \
                          if (any(l.strip().startswith(i) for i in ('"UTM-X', '"UTM-Y', 'Date')) and len(l.strip().rsplit('\t', maxsplit=1)) == 2))

            cpt[key] = {}
            cpt[key]['CPT_data'] = temp

            for i in d[key]:
                if i.startswith('"UTM-X'):
                    cpt[key]['UTM-X'] = int(d[key][i])
                elif i.startswith('"UTM-Y'):
                    cpt[key]['UTM-Y'] = int(d[key][i])

                elif i.startswith('Date'):
                    cpt[key]['Date'] = datetime.datetime.strptime(d[key][i], '%m/%d/%Y')


    for i in range(len(names)):
        cpt[names[i]]['Lat'], cpt[names[i]]['Lon'] = utm.to_latlon(cpt[names[i]]['UTM-X'], cpt[names[i]]['UTM-Y'], 10,
                                                                   northern=True)
    return cpt


In [8]:
cpt = setup_cpt('./USGS_CPT_data/')

In [10]:
keys = list(cpt.keys())
lats = []
lons = []
for key in cpt:
    lats.append(cpt[key]['Lat'])
    lons.append(cpt[key]['Lon'])

In [12]:
site_file = "./all_alameda_sites.csv"
with open('./event_save_all_alameda.pickle','rb') as handle:
    event_set = pickle.load(handle)
    pgas = {}
pgaZ = np.genfromtxt('./pypsha_pgas.csv', delimiter=',')
data = pd.read_csv(site_file,index_col=0)
X = data['x'] # or lon
Y = data['y'] # or lat

In [16]:
nsim = len(pgaZ)
grid = pd.DataFrame(columns = ['pga','lat','lon'])
grid['lat'] = lats
grid['lon'] = lons
grid.index = keys
for sim in range(nsim):
    Z = []
    for i in range(pgaZ.shape[1]): #pga at each location
        Z.append(pgaZ[sim,i])
    Z = np.array(Z)
    z_t = NNR(np.array([lons,lats]).T,np.array([X,Y]).T,Z,sample_size = -1, n_neighbors = 4, weight = 'distance2')
    grid['pga'] = z_t
    gdf = gpd.GeoDataFrame(grid, geometry=gpd.points_from_xy(grid['lon'], grid['lat']),crs = 'EPSG:4326')
    if sim == 0:
        for i in keys:
            pgas[i] = []
    for i in keys:
        pgas[i].append(gdf['pga'][i])
        
for i in keys:
    pgas[i] = np.array(pgas[i])

In [19]:
pd.DataFrame(pgas).to_csv('./pgas_cpt.csv')